In [47]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display


API_KEY = "f2a0fbf712db401c8743e0d5b232d3ce"
HEADERS = {"X-Auth-Token": API_KEY}
BASE_URL = "https://api.football-data.org/v4"

def fetch_data(endpoint, params=None):
    if params is None:
        params = {}
    response = requests.get(f"{BASE_URL}/{endpoint}", headers=HEADERS, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching {endpoint}: {response.status_code}")
        return {"data": []}


In [48]:
def get_upcoming_games():
    today = datetime.now().strftime("%Y-%m-%d")
    next_week = (datetime.now() + timedelta(days=7)).strftime("%Y-%m-%d")
    params = {"dateFrom": today, "dateTo": next_week}
    data = fetch_data("competitions/PL/matches", params)
    matches = data["matches"]
    if not matches:
        return pd.DataFrame()
    df = pd.DataFrame([{
        "Match": f"{match['homeTeam']['name']} vs {match['awayTeam']['name']}",
        "Date": match['utcDate'],
        "Status": match['status']
    } for match in matches if match['status'] in ["SCHEDULED", "TIMED"]])
    return df.sort_values("Date")
print("Upcoming Games (Next 7 Days) - Premier League")
upcoming = get_upcoming_games()
display(upcoming)

Upcoming Games (Next 7 Days) - Premier League


,Match,Date,Status
0,Arsenal FC vs Fulham FC,2025-04-01T18:45:00Z,TIMED
1,Wolverhampton Wanderers FC vs West Ham United FC,2025-04-01T18:45:00Z,TIMED
2,Nottingham Forest FC vs Manchester United FC,2025-04-01T19:00:00Z,TIMED
3,AFC Bournemouth vs Ipswich Town FC,2025-04-02T18:45:00Z,TIMED
4,Brighton & Hove Albion FC vs Aston Villa FC,2025-04-02T18:45:00Z,TIMED
5,Manchester City FC vs Leicester City FC,2025-04-02T18:45:00Z,TIMED
6,Newcastle United FC vs Brentford FC,2025-04-02T18:45:00Z,TIMED
7,Southampton FC vs Crystal Palace FC,2025-04-02T18:45:00Z,TIMED
8,Liverpool FC vs Everton FC,2025-04-02T19:00:00Z,TIMED


In [ ]:
def get_standings():
    data = fetch_data("competitions/PL/standings")
    standings = data["standings"][0]["table"]  # Total standings
    if not standings:
        return pd.DataFrame()
    df = pd.DataFrame([{
        "Position": team["position"],
        "Team": team["team"]["name"],
        "Points": team["points"],
        "Played": team["playedGames"]
    } for team in standings])
    return df
print("\nPremier League Standings")
standings = get_standings()
display(standings)


Premier League Standings


,Position,Team,Points,Played
0,1,Liverpool FC,70,29
1,2,Arsenal FC,58,29
2,3,Nottingham Forest FC,54,29
3,4,Chelsea FC,49,29
4,5,Manchester City FC,48,29
5,6,Newcastle United FC,47,28
6,7,Brighton & Hove Albion FC,47,29
7,8,Fulham FC,45,29
8,9,Aston Villa FC,45,29
9,10,AFC Bournemouth,44,29


In [ ]:
def get_live_matches():
    data = fetch_data("matches", {"status": "LIVE"})
    matches = [m for m in data["matches"] if m["competition"]["id"] == 2021]  # PL ID
    if not matches:
        return pd.DataFrame()
    df = pd.DataFrame([{
        "Match": f"{match['homeTeam']['name']} vs {match['awayTeam']['name']}",
        "Score": f"{match['score']['fullTime']['home']} - {match['score']['fullTime']['away']}",
        "Minute": match['score'].get('duration', 'N/A')
    } for match in matches])
    return df
print("\nLive Matches (Snapshot)")
live_matches = get_live_matches()
display(live_matches)


Live Matches (Snapshot)


""
